In [1]:
import torch

In [2]:
x = torch.rand(7,2)

In [3]:
print(x)

tensor([[0.5124, 0.7670],
        [0.5263, 0.6204],
        [0.3476, 0.9393],
        [0.6033, 0.4477],
        [0.0305, 0.4709],
        [0.2153, 0.3563],
        [0.3896, 0.0352]])


In [1]:
import torch

In [2]:
torch.cuda.is_available()

True